In [2]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import lxml
import sqlite3 as lite
import pandas as pd
# from pymongo import MongoClient

# Web Scraping BillBoard 200 2017-2018

## With Selenium

In [2]:
ch_driver = webdriver.Chrome('C:/Users/beave/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Python 3.7/chromedriver.exe')

In [3]:
date = '2019-01-19'
billboard_url = 'https://www.billboard.com/charts/billboard-200/'
ch_driver.get(billboard_url + date)

In [6]:
# for first song
first_song = ch_driver.find_element_by_class_name('chart-number-one__details')
rank = '1'
title = first_song.find_element_by_class_name('chart-number-one__title').text
artist = first_song.find_element_by_class_name('chart-number-one__artist').text
print(rank, title, artist)

1 Hoodie SZN A Boogie Wit da Hoodie


In [7]:
# for the rest
chart_list = ch_driver.find_elements_by_class_name('chart-list-item')
rank = chart_list[0].get_attribute('data-rank')
title = chart_list[0].get_attribute('data-title')
artist = chart_list[0].get_attribute('data-artist')

print(rank, title, artist)

2 I Am > I Was 21 Savage


In [9]:
result_list = []
i = 0

first_song = ch_driver.find_element_by_class_name('chart-number-one__details')
rank = '1'
title = first_song.find_element_by_class_name('chart-number-one__title').text
artist = first_song.find_element_by_class_name('chart-number-one__artist').text

result_list.append([rank, title, artist])

chart_list = ch_driver.find_elements_by_class_name('chart-list-item')
for item in chart_list:
    rank = item.get_attribute('data-rank')
    title = item.get_attribute('data-title')
    artist = item.get_attribute('data-artist')
    
    result_list.append([rank, title, artist])

# for check
#result_list



[['1', 'Hoodie SZN', 'A Boogie Wit da Hoodie'],
 ['2', 'I Am > I Was', '21 Savage'],
 ['3', 'Spider-Man: Into The Spider-Verse', 'Soundtrack'],
 ['4', 'Championships', 'Meek Mill'],
 ['5', 'beerbongs & bentleys', 'Post Malone'],
 ['6', 'ASTROWORLD', 'Travis Scott'],
 ['7', 'A Star Is Born (Soundtrack)', 'Lady Gaga & Bradley Cooper'],
 ['8', 'Scorpion', 'Drake'],
 ['9', 'Dying To Live', 'Kodak Black'],
 ['10', 'Greatest Hits I II & III: The Platinum Collection', 'Queen'],
 ['11', 'The Greatest Showman', 'Soundtrack'],
 ['12', 'X 100PRE', 'Bad Bunny'],
 ['13', 'Goodbye & Good Riddance', 'Juice WRLD'],
 ['14', 'Invasion Of Privacy', 'Cardi B'],
 ['15', 'Realer', 'YoungBoy Never Broke Again'],
 ['16', 'Drip Harder', 'Lil Baby & Gunna'],
 ['17', "This One's For You", 'Luke Combs'],
 ['18', 'Sweetener', 'Ariana Grande'],
 ['19', 'Dont Smile At Me', 'Billie Eilish'],
 ['20', '?', 'XXXTENTACION'],
 ['21', 'Street Gossip', 'Lil Baby'],
 ['22', 'Origins', 'Imagine Dragons'],
 ['23', 'Stoney', 'P

## With BeautifulSoup

In [3]:
date = '2019-01-19'
billboard_url = 'https://www.billboard.com/charts/billboard-200/'

response = requests.get(billboard_url + date)
bb_html = response.text
bb_soup = BeautifulSoup(bb_html, 'lxml')

In [4]:
# for the first-place ranked song
rank = '1'
title = bb_soup.find('div', attrs={'class': 'chart-number-one__title'}).text
artist = bb_soup.find('div', attrs={'class': 'chart-number-one__artist'}).text.strip()

print(rank, title, artist, sep=", ")

1, Hoodie SZN, A Boogie Wit da Hoodie


In [5]:
# for the rest
bb_divs = bb_soup.find_all('div', attrs={'class':'chart-list-item'})

rank = bb_divs[0].attrs['data-rank']
title = bb_divs[0].attrs['data-title']
artist = bb_divs[0].attrs['data-artist']

print(rank, title, artist, sep=",")

2,I Am > I Was,21 Savage


In [7]:
def billboard_scraper(year, month):
    date = str(year) + '-' + month + '-15' 
    billboard_url = 'https://www.billboard.com/charts/billboard-200/'
    response = requests.get(billboard_url + date)
    bb_html = response.text
    bb_soup = BeautifulSoup(bb_html, 'lxml')
    
    result_list = []
    # for the chart number one
    rank = '1'
    title = bb_soup.find('div', attrs={'class': 'chart-number-one__title'}).text
    artist = bb_soup.find('div', attrs={'class': 'chart-number-one__artist'}).text.strip()
    result_list.append((date, rank, title, artist))

    #for the rest
    bb_divs = bb_soup.find_all('div', attrs={'class':'chart-list-item'})

    for item in bb_divs:
        rank = item.attrs['data-rank']
        title = item.attrs['data-title']
        artist = item.attrs['data-artist']

        result_list.append((date, rank, title, artist))

    return result_list

In [8]:
def get_all_billboard():
    month = '01'
    month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
    billboard_list = []

    for year in range(2017, 2018+1):
        for month in month_list:
            billboard_list.extend(billboard_scraper(year, month))

    return billboard_list

# ('2017-01-15', 'rank', title', 'artist')

# Make a Table for Billboard Chart 

- Create chart.db with sqlite3<br/>
`.open chart_research.db`<br/>
- Move Database to the project folder

In [9]:
result_list = get_all_billboard()
db = lite.connect('./chart_research.db')

In [18]:
pd.options.display.max_colwidth = 300

In [10]:
len(result_list)

4800

In [11]:
result_list[0]

('2017-01-15', '1', 'Starboy', 'The Weeknd')

In [15]:
query_create_table = """
CREATE TABLE IF NOT EXISTS
Billboard
("Date" TEXT,
"Rank" TEXT,
"Title" TEXT,
"Artist" TEXT);
"""

query_insert_data = """
INSERT INTO Billboard VALUES(?,?,?,?);
"""

with db:
    cur = db.cursor()
    cur.execute(query_create_table)
    for row in result_list:
        cur.execute(query_insert_data,row)

In [29]:
query = """
SELECT 
    substr(date, 1, 7) "Month",
    Rank,
    Title,
    Artist
FROM
    Billboard
;
"""

pd.read_sql(query, db).head()

,Month,Rank,Title,Artist
0,2017-01,1,Starboy,The Weeknd
1,2017-01,2,Moana,Soundtrack
2,2017-01,3,24K Magic,Bruno Mars
3,2017-01,4,Views,Drake
4,2017-01,5,4 Your Eyez Only,J. Cole
